In [75]:
import re
import json
import time
import nltk
import numpy as np
import pprint
import string
import typing
import pandas as pd
import collections


nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/makroguzov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [76]:
DATA_PATH = './movies'

In [77]:
M_PATH = f'{DATA_PATH}/movies.json'  #
U_PATH = f'{DATA_PATH}/users'  #

In [78]:
def import_movies(m_path):
    """Генератор фильмов. Возвращаю словарик с фильмами"""
    with open(m_path, 'r', encoding='utf-8') as f:
        data = f.read()
        for movie in json.loads(data).values():
            if movie['description']:
                yield {
                    'id': movie['id'],
                    'name': movie['name'],
                    'russian': movie['russian'],
                    'score': movie['score'],
                    'description': movie['description'],
                }

In [79]:
def import_genres(m_path):
    """Генератор собирающий все жанры по фильмам"""
    with open(m_path, 'r', encoding='utf-8') as f:
        data = f.read()
        for movie in json.loads(data).values():
            for genre in movie['genres']:
                genre['movie_id'] = movie['id']
                yield genre

In [80]:
MOVIES = pd.DataFrame(import_movies(M_PATH))
MOVIES.head()

,id,name,russian,score,description
0,4884,Tales of the Abyss,Сказания Бездны,7.29,"Люк фон Фабр, избалованный отпрыск благородных..."
1,4896,Umineko no Naku Koro ni,Когда плачут чайки,7.09,"Действие происходит на острове Роккэнджима, пр..."
2,4898,Kuroshitsuji,Тёмный дворецкий,7.7,События происходят в Англии викторианской эпох...
3,4975,ChaoS;HEAd,Вершина хаоса,6.33,"Сибуя, Япония, 2008 год. Такуми Нисидзё [西條 拓巳..."
4,4999,Asu no Yoichi!,Ёити завтрашнего дня,6.74,После многих лет тренировок и самосовершенство...


In [81]:
GENRES = pd.DataFrame(import_genres(M_PATH))
GENRES.head()

,id,name,russian,kind,movie_id
0,2,Adventure,Приключения,anime,4884
1,8,Drama,Драма,anime,4884
2,10,Fantasy,Фэнтези,anime,4884
3,14,Horror,Ужасы,anime,4896
4,7,Mystery,Детектив,anime,4896


In [82]:
TEXT_COLUMNS = ['russian', 'description']
TARGET_COLUMN = 'score'

In [83]:
from nltk.corpus import stopwords
from string import punctuation

RUSSIAN_STOP_WORDS = stopwords.words('russian')
REMOVE_OTHERS_PATTERN = re.compile(r'[^А-яЁё]+')
TRANSLATE_PATTERN = re.compile(r'[Ёё]+')
TOKENIZER = nltk.tokenize.WordPunctTokenizer()
PUNCTUATION = set(punctuation)


def normalize(text: str):
    text = text.lower()
    text = REMOVE_OTHERS_PATTERN.sub(' ', text).strip()
    text = TRANSLATE_PATTERN.sub('е', text)
    text = TOKENIZER.tokenize(text)
    text = [
        word for word in text
        if word not in RUSSIAN_STOP_WORDS and word not in PUNCTUATION
    ]
    return ' '.join(text)

In [84]:
MOVIES[TEXT_COLUMNS] = MOVIES[TEXT_COLUMNS].applymap(normalize)
MOVIES.head()

,id,name,russian,score,description
0,4884,Tales of the Abyss,сказания бездны,7.29,люк фон фабр избалованный отпрыск благородных ...
1,4896,Umineko no Naku Koro ni,плачут чайки,7.09,действие происходит острове роккэнджима принад...
2,4898,Kuroshitsuji,темный дворецкий,7.7,события происходят англии викторианской эпохи ...
3,4975,ChaoS;HEAd,вершина хаоса,6.33,сибуя япония год такуми нисидзе ученик средней...
4,4999,Asu no Yoichi!,еити завтрашнего дня,6.74,многих лет тренировок самосовершенствования об...


In [85]:
TOKENS_COUNTER = collections.Counter()
for _, row in MOVIES[TEXT_COLUMNS].iterrows():
    for string in row:
        TOKENS_COUNTER.update(string.split())

print('\n'.join(map(str, TOKENS_COUNTER.most_common(5))))
print('...')
print('\n'.join(map(str, TOKENS_COUNTER.most_common()[-3:])))

('это', 1536)
('однако', 1226)
('время', 1107)
('который', 1102)
('аниме', 1047)
...
('чудак', 1)
('живее', 1)
('чудаковатого', 1)


In [86]:
UNK, PAD = "UNK", "PAD"
TOKENS = [UNK, PAD] + sorted(TOKENS_COUNTER.keys())

In [87]:
TOKEN_IDS = {token: idx for idx, token in enumerate(TOKENS)}
TOKEN_IDS

{'UNK': 0,
 'PAD': 1,
 'аарон': 2,
 'аарона': 3,
 'аб': 4,
 'абараи': 5,
 'абасири': 6,
 'аббревиатуру': 7,
 'абель': 8,
 'аберрации': 9,
 'абигейл': 10,
 'абиру': 11,
 'абитуриент': 12,
 'або': 13,
 'аборигенов': 14,
 'абсолютная': 15,
 'абсолютно': 16,
 'абсолютного': 17,
 'абсолютной': 18,
 'абсолютную': 19,
 'абсолютный': 20,
 'абсолютным': 21,
 'абсолютных': 22,
 'абстрактном': 23,
 'абстрактный': 24,
 'абсурд': 25,
 'абсурда': 26,
 'абсурдная': 27,
 'абсурдно': 28,
 'абсурдного': 29,
 'абсурдное': 30,
 'абсурдной': 31,
 'абсурдности': 32,
 'абсурдные': 33,
 'абсурдным': 34,
 'абсурдными': 35,
 'абсурдных': 36,
 'абы': 37,
 'абэ': 38,
 'абэно': 39,
 'абэнобаси': 40,
 'авадзи': 41,
 'авакадо': 42,
 'авангард': 43,
 'авангарда': 44,
 'авангардов': 45,
 'авангардом': 46,
 'авангарды': 47,
 'авантюра': 48,
 'авантюре': 49,
 'авантюрист': 50,
 'авантюриста': 51,
 'авантюристах': 52,
 'авантюристка': 53,
 'авантюристкам': 54,
 'авантюристов': 55,
 'авантюристом': 56,
 'авантюристы': 57,

In [88]:
UNK_ID, PAD_ID = map(TOKEN_IDS.get, [UNK, PAD])


def matrix(seq: typing.Sequence[str], max_len=None):
    seq = [s.split() for s in seq]
    max_len = min(max(map(len, seq)), max_len or float('inf'))

    matrix_shape = len(seq), max_len
    matrix = np.full(matrix_shape, np.int32(PAD_ID))

    for i, tokens in enumerate(seq):
        row = [TOKEN_IDS.get(word, UNK_ID) for word in tokens[:max_len]]
        matrix[i, :len(row)] = row

    return matrix

In [89]:
print("Названия фильмов:")
print('\n'.join(MOVIES["russian"][:3].values), end='\n\n')
print("Названия фильмов в виде матрицы:")
print(matrix(MOVIES["russian"][:3]))

Названия фильмов:
сказания бездны
плачут чайки
темный дворецкий

Названия фильмов в виде матрицы:
[[49829  2474]
 [36773 61459]
 [55118 10584]]


In [90]:
from sklearn.model_selection import train_test_split

DATA_TRAIN, DATA_VALID = train_test_split(MOVIES,
                                          test_size=0.2,
                                          random_state=42)

DATA_TRAIN.index = range(len(DATA_TRAIN))
DATA_VALID.index = range(len(DATA_VALID))

print("Train size = ", len(DATA_TRAIN))
print("Validation size = ", len(DATA_VALID))

Train size =  5468
Validation size =  1368


In [91]:
DATA_TRAIN.head()

,id,name,russian,score,description
0,39017,Kyokou Suiri,ложные выводы,6.92,детстве котоко иванага унесена екаями сверхъес...
1,7805,Baka to Test to Shoukanjuu: Mondai - Christmas...,дурни тесты аватары рождество,6.64,персонажи дурней тестов аватаров расскажут зри...
2,40046,Id:Invaded,вторжение,7.86,кура организация работает расследованием прест...
3,10460,Kimi to Boku.,,7.68,история закручивается вокруг четырех подростко...
4,33489,Little Witch Academia (TV),академия ведьмочек,7.84,акко кагари детстве попала магическое шоу ведь...


In [92]:
DATA_VALID.head()

,id,name,russian,score,description
0,31452,Norn9: Norn+Nonet,норн норн нонет,6.56,недалекое будущее ученик начальной школы сорат...
1,40776,Haikyuu!!: To the Top 2nd Season,волейбол вершине,8.54,яростная борьба вершину среди волейбольных ком...
2,4722,Skip Beat!,сдавайся,8.1,история рассказывает кеко могами летней девушк...
3,30193,Eun-sil-i,дорогой,5.97,ин хе сон ми долгого отсутствия приезжают наве...
4,35347,Kemono Friends: Bus-teki,друзья зверушки эпизод,6.59,битвы гигантским лазурником белолицая совка фи...


In [93]:
def get_batch(data, max_len=None):
    batch = {
        'russian': matrix(data['russian'].values, max_len),
        'description': matrix(data['description'].values, max_len),
    }

    if TARGET_COLUMN in data.columns:
        batch[TARGET_COLUMN] = data[TARGET_COLUMN].values

    return batch


test = get_batch(MOVIES[:5], 10)
test

{'russian': array([[49829,  2474,     1],
        [36773, 61459,     1],
        [55118, 10584,     1],
        [ 4928, 60063,     1],
        [13027, 13986, 11723]], dtype=int32),
 'description': array([[23612, 59458, 58911, 16500, 34537,  3243, 21571, 20927, 19465,
         40220],
        [10800, 43247, 33818, 47208, 42126, 35020,  3422, 49048, 58859,
         32009],
        [50999, 43249,  1041,  5449, 63430, 46249, 32428, 40051, 48615,
          3318],
        [49354, 64111,  9200, 54616, 30438, 58762, 52556, 62526, 51082,
         15563],
        [25718, 22779, 56224, 48011, 31302,  3494, 17709, 22808, 13027,
         18885]], dtype=int32),
 'score': array(['7.29', '7.09', '7.7', '6.33', '6.74'], dtype=object)}

In [94]:
def iterate_minibatches(data,
                        batch_size=256,
                        shuffle=True,
                        cycle=False,
                        **kwargs):
    while True:
        indices = np.arange(len(data))
        if shuffle:
            indices = np.random.permutation(indices)

        for start in range(0, len(indices), batch_size):
            batch_ = get_batch(data.iloc[indices[start:start + batch_size]],
                               **kwargs)
            yield batch_, np.array(batch_.pop(TARGET_COLUMN), dtype=np.float16)

        if not cycle: break


test = iterate_minibatches(MOVIES, 3)
b, t = next(test)

print('Batch:')
pprint.pprint(b)
print('Tatger:')
pprint.pprint(t)

Batch:
{'description': array([[21398, 36170, 50997, 35711,  6807, 63409,  1127, 30763, 30763,
        30715,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1],
       [50997, 41056, 48883, 58249,  1265, 39191,  8057, 20832, 53595,
         6997, 42718, 25837, 12539, 13035,  2730, 52358, 57967,  5005,
        64118,  1581, 49456, 33782, 26100, 45369, 29939,  1265, 49446,
        20832],
       [19152, 18517, 61066, 11954, 34238, 59257,  8372, 20014,   886,
        27161, 30666, 55713, 24765, 30666,  7203, 30647, 12645, 54564,
        58220, 44515, 58346, 43867, 13832,     1,     1,     1,     1,
            1]], dtype=int32),
 'russian': array([[30763, 30763, 30715, 46474],
       [58249, 55842,     1,     1],
       [18517, 61066, 11954,     1]], dtype=int32)}
Tatger:
array([6.15, 7.9 , 4.75], dtype=float16)


In [95]:
import torch
import torch.nn as nn


class MaxPooling(nn.Module):

    def __init__(self, dim=-1):
        super().__init__()
        self.dim = dim

    def forward(self, x):
        return x.max(dim=self.dim)[0]


class Model(nn.Module):

    def __init__(self,
                 n_tokens: int,
                 concat_number_of_features: int,
                 hid_size: int = 69) -> None:
        super().__init__()
        # Russian title Embanding
        self.t_embedding = nn.Embedding(num_embeddings=n_tokens,
                                        embedding_dim=hid_size)
        # Full description Embanding
        self.f_embedding = nn.Embedding(num_embeddings=n_tokens,
                                        embedding_dim=hid_size)

        self.conv1 = nn.Conv1d(hid_size, hid_size, kernel_size=3, padding=1)
        self.dense = nn.Linear(hid_size, hid_size)
        self.pool = MaxPooling()

        self.inter_dense = nn.Linear(in_features=concat_number_of_features,
                                     out_features=2 * hid_size)
        self.final_dense = nn.Linear(in_features=2 * hid_size, out_features=1)

    def forward(self, input: tuple):
        i1, i2 = input

        title = self.t_embedding(i1).permute((0, 2, 1))
        title = self.conv1(title)
        title = self.pool(title)
        title = nn.ReLU()(title)
        title = self.dense(title)

        fulld = self.f_embedding(i2).permute((0, 2, 1))
        fulld = self.conv1(fulld)
        fulld = self.pool(fulld)
        fulld = nn.ReLU()(fulld)
        fulld = self.dense(fulld)

        conct_arrays = [
            title.view(title.size(0), -1),
            fulld.view(fulld.size(0), -1),
        ]

        out = nn.Sequential(self.inter_dense, nn.ReLU(), self.final_dense)

        return out(torch.cat(conct_arrays, dim=1))


In [96]:


model = Model(n_tokens=len(TOKEN_IDS),
              concat_number_of_features=69 * 2,
              hid_size=69)
model

Model(
  (t_embedding): Embedding(64251, 69)
  (f_embedding): Embedding(64251, 69)
  (conv1): Conv1d(69, 69, kernel_size=(3,), stride=(1,), padding=(1,))
  (dense): Linear(in_features=69, out_features=69, bias=True)
  (pool): MaxPooling()
  (inter_dense): Linear(in_features=138, out_features=138, bias=True)
  (final_dense): Linear(in_features=138, out_features=1, bias=True)
)

In [100]:
len(TOKEN_IDS)

64251

In [97]:
N_EPOCHS = 10
BATCH_SIZE = 256
OPTIMIZATOR = torch.optim.Adam(model.parameters(), lr=1e-3)


def train_one_epoch(batch, target, optimizator):
    title = torch.tensor(batch['russian'], dtype=torch.int64)
    descr = torch.tensor(batch['description'], dtype=torch.int64)
    target = torch.tensor(target, dtype=torch.float32)

    prediction = model((title, descr))

    loss = torch.mean((prediction - target)**2)
    loss.backward()

    optimizator.step()
    optimizator.zero_grad()

    return loss.data.numpy()


print("Start training process:")

for epoch in range(1, N_EPOCHS + 1):
    train_start_time = time.time()
    train_loss = 0
    train_batches = 0
    model.train()

    for batch, target in iterate_minibatches(DATA_TRAIN, BATCH_SIZE):
        train_loss += train_one_epoch(batch, target, OPTIMIZATOR)
        train_batches += 1

    # Время тренировки одного поколения
    duration = time.time() - train_start_time

    print(
        f"Epoch: {epoch:02} | Time: {int(duration / 60)}:m {int(duration - (int(duration / 60) * 60))}:s"
    )
    print(f"Train Loss: {(train_loss / train_batches):.3f}")
    print()

Start training process:
Epoch: 01 | Time: 0:m 3:s
Train Loss: 15.183

Epoch: 02 | Time: 0:m 3:s
Train Loss: 3.863

Epoch: 03 | Time: 0:m 3:s
Train Loss: 3.256

Epoch: 04 | Time: 0:m 3:s
Train Loss: 3.127

Epoch: 05 | Time: 0:m 3:s
Train Loss: 3.100

Epoch: 06 | Time: 0:m 3:s
Train Loss: 3.020

Epoch: 07 | Time: 0:m 3:s
Train Loss: 3.016

Epoch: 08 | Time: 0:m 3:s
Train Loss: 3.040

Epoch: 09 | Time: 0:m 3:s
Train Loss: 3.073

Epoch: 10 | Time: 0:m 3:s
Train Loss: 3.076



In [98]:
MODEL_PATH = '/Users/makroguzov/Desktop/animizer/animizer/model/model'
TOKEN_PATH = '/Users/makroguzov/Desktop/animizer/animizer/model/token.txt'

In [99]:
json.dump(TOKEN_IDS, open(TOKEN_PATH, 'w', encoding='utf-8'))
torch.save(model.state_dict(), MODEL_PATH)


In [102]:
MOVIES.iloc[0]

id                                                          4884
name                                          Tales of the Abyss
russian                                          сказания бездны
score                                                       7.29
description    люк фон фабр избалованный отпрыск благородных ...
Name: 0, dtype: object

In [107]:
batch = MOVIES.iloc[0]
print(batch)

title = torch.tensor(matrix([batch['russian']]), dtype=torch.int64)
descr = torch.tensor(matrix([batch['description']]), dtype=torch.int64)

model((
    title,
    descr,
))


id                                                          4884
name                                          Tales of the Abyss
russian                                          сказания бездны
score                                                       7.29
description    люк фон фабр избалованный отпрыск благородных ...
Name: 0, dtype: object


tensor([[3.5331]], grad_fn=<AddmmBackward0>)